# Nevezett Entitás Felismerés (NER)

Ez a jegyzetfüzet az [AI for Beginners Curriculum](http://aka.ms/ai-beginners) része.

Ebben a példában megtanuljuk, hogyan képezzünk ki egy NER modellt a [Annotated Corpus for Named Entity Recognition](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus) adathalmazon, amely elérhető a Kaggle platformon. Mielőtt folytatnánk, kérjük, töltsd le a [ner_dataset.csv](https://www.kaggle.com/datasets/abhinavwalia95/entity-annotated-corpus?resource=download&select=ner_dataset.csv) fájlt az aktuális könyvtárba.


In [62]:
import pandas as pd
from tensorflow import keras
import numpy as np

## Az adathalmaz előkészítése

Először beolvassuk az adathalmazt egy dataframe-be. Ha többet szeretnél megtudni a Pandas használatáról, látogass el egy [adatfeldolgozási leckére](https://github.com/microsoft/Data-Science-For-Beginners/tree/main/2-Working-With-Data/07-python) a [Data Science for Beginners](http://aka.ms/datascience-beginners) oldalán.


In [3]:
df = pd.read_csv('ner_dataset.csv',encoding='unicode-escape')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Szerezzünk egyedi címkéket, és hozzunk létre keresési szótárakat, amelyeket használhatunk a címkék osztályszámokká történő átalakításához:


In [4]:
tags = df.Tag.unique()
tags

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [8]:
id2tag = dict(enumerate(tags))
tag2id = { v : k for k,v in id2tag.items() }

id2tag[0]

'O'

Most már a szókincs esetében is ugyanezt kell tennünk. Az egyszerűség kedvéért olyan szókincset fogunk létrehozni, amely nem veszi figyelembe a szavak gyakoriságát; a valós életben érdemes lehet a Keras vektorizálót használni, és korlátozni a szavak számát.


In [14]:
vocab = set(df['Word'].apply(lambda x: x.lower()))
id2word = { i+1 : v for i,v in enumerate(vocab) }
id2word[0] = '<UNK>'
vocab.add('<UNK>')
word2id = { v : k for k,v in id2word.items() }

Szükségünk van egy mondatokból álló adatállomány létrehozására a tanításhoz. Haladjunk végig az eredeti adatállományon, és válasszuk szét az összes egyedi mondatot `X` (szavak listája) és `Y` (tokenek listája) formájában:


In [41]:
X,Y = [],[]
s,t = [],[]
for i,row in df[['Sentence #','Word','Tag']].iterrows():
    if pd.isna(row['Sentence #']):
        s.append(row['Word'])
        t.append(row['Tag'])
    else:
        if len(s)>0:
            X.append(s)
            Y.append(t)
        s,t = [row['Word']],[row['Tag']]
X.append(s)
Y.append(t)


In [93]:
def vectorize(seq):
    return [word2id[x.lower()] for x in seq]

def tagify(seq):
    return [tag2id[x] for x in seq]

Xv = list(map(vectorize,X))
Yv = list(map(tagify,Y))

Xv[0], Yv[0]

([10386,
  23515,
  4134,
  29620,
  7954,
  13583,
  21193,
  12222,
  27322,
  18258,
  5815,
  15880,
  5355,
  25242,
  31327,
  18258,
  27067,
  23515,
  26444,
  14412,
  358,
  26551,
  5011,
  30558],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])

Az egyszerűség kedvéért az összes mondatot 0 tokenekkel egészítjük ki a maximális hosszúságig. A valóságban érdemesebb lehet egy okosabb stratégiát alkalmazni, és csak egy minibatch-en belül kitölteni a szekvenciákat.


In [51]:
X_data = keras.preprocessing.sequence.pad_sequences(Xv,padding='post')
Y_data = keras.preprocessing.sequence.pad_sequences(Yv,padding='post')

## Tokenosztályozó hálózat meghatározása

Két rétegű, kétirányú LSTM hálózatot fogunk használni a tokenek osztályozásához. Annak érdekében, hogy a sűrű osztályozót alkalmazzuk az utolsó LSTM réteg minden egyes kimenetére, a `TimeDistributed` konstrukciót fogjuk használni, amely ugyanazt a sűrű réteget replikálja az LSTM minden egyes lépésének kimenetére:


In [94]:
maxlen = X_data.shape[1]
vocab_size = len(vocab)
num_tags = len(tags)
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, 300, input_length=maxlen),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(units=100, activation='tanh', return_sequences=True)),
    keras.layers.TimeDistributed(keras.layers.Dense(num_tags, activation='softmax'))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 104, 300)          9545400   
                                                                 
 bidirectional_6 (Bidirectio  (None, 104, 200)         320800    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 104, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed_3 (TimeDis  (None, 104, 17)          3417      
 tributed)                                                       
                                                                 
Total params: 10,110,417
Trainable params: 10,110,417
Non-trainable params: 0
__________________________________________

Fontos megjegyezni, hogy itt kifejezetten megadjuk a `maxlen` értéket az adatállományunkhoz - ha azt szeretnénk, hogy a hálózat változó hosszúságú szekvenciákat is kezeljen, akkor egy kicsit okosabban kell eljárnunk a hálózat definiálásakor.

Most tanítsuk be a modellt. A gyorsaság érdekében csak egy epoch-ra fogunk tanítani, de érdemes lehet hosszabb ideig is próbálkozni. Emellett érdemes lehet az adatállomány egy részét elkülöníteni tanító adatállományként, hogy megfigyelhessük az érvényességi pontosságot.


In [57]:
model.fit(X_data,Y_data)

1499/1499 [==============================] - 740s 488ms/step - loss: 0.0667 - acc: 0.9841


## Az eredmény tesztelése

Nézzük meg, hogyan működik az entitásfelismerő modellünk egy mintamondaton:


In [91]:
sent = 'John Smith went to Paris to attend a conference in cancer development institute'
words = sent.lower().split()
v = keras.preprocessing.sequence.pad_sequences([[word2id[x] for x in words]],padding='post',maxlen=maxlen)
res = model(v)[0]

In [92]:
r = np.argmax(res.numpy(),axis=1)
for i,w in zip(r,words):
    print(f"{w} -> {id2tag[i]}")

john -> B-per
smith -> I-per
went -> O
to -> O
paris -> B-geo
to -> O
attend -> O
a -> O
conference -> O
in -> O
cancer -> B-org
development -> I-org
institute -> I-org


## Összegzés

Még egy egyszerű LSTM modell is elfogadható eredményeket mutat az NER esetében. Azonban, ha sokkal jobb eredményeket szeretnél elérni, érdemes nagy, előre betanított nyelvi modelleket használni, mint például a BERT. A BERT betanítását NER-re a Huggingface Transformers könyvtár segítségével [itt](https://huggingface.co/course/chapter7/2?fw=pt) találod leírva.



---

**Felelősség kizárása**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
